In [ ]:
import math
from matplotlib import pyplot as plt
import numpy as np
import numpy.random as nprand
import pandas as pd
import random
import scipy.stats as spstats
%matplotlib inline

from loomio import *
from socialchoice import *
from timeseries import *

In [ ]:
df_score, proposal_map, proposal_rev_map = load_loomio_score('results/results_2_3.tsv')
stage_profiles = [
    Profile.from_score(df_score[df_score.stage == stage])
    for stage in sorted(set(df_score.stage))]

Methods = [Majority, Borda, Tideman]
print('stage\t' + '\t'.join([M.__name__ for M in Methods]))
for stage, profile in enumerate(stage_profiles):
    winners = [repr(M(profile).social_choice()) for M in Methods]
    print(str(stage) + '\t' + '\t'.join(winners))


## Group-Level Results

In [ ]:
df_score, proposal_map, proposal_rev_map = load_loomio_score('results/results_2_3.tsv')

treatments = {
    1: "Single Group",
    2: "Random Pod",
}

for treatment in [1, 2]:
    print("Treatment: {}".format(treatments[treatment]))
    stage_profiles = [
        Profile.from_score(df_score[(df_score.stage == stage) & (df_score.treatment == treatment)])
        for stage in sorted(set(df_score.stage))]

    Methods = [Majority, Borda, Tideman]
    print('stage\t' + '\t'.join([M.__name__ for M in Methods]))
    for stage, profile in enumerate(stage_profiles):
        winners = [repr(M(profile).social_choice()) for M in Methods]
        print(str(stage) + '\t' + '\t'.join(winners))
    print("")


### Group-Level Agreement

In [ ]:
control_profiles = [
    Profile.from_score(df_score[(df_score.stage == stage) & (df_score.treatment == 1)])
    for stage in sorted(set(df_score.stage))]

random_profiles = [
    Profile.from_score(df_score[(df_score.stage == stage) & (df_score.treatment == 2)])
    for stage in sorted(set(df_score.stage))]

bootstrap_runs = 10

kendall_control = TimeSeriesResult()
spearman_control = TimeSeriesResult()
tideman_control = TimeSeriesResult()
print('Single Group')
print('stage\tkendall\tspearman\ttideman')
for stage, profile in enumerate(control_profiles):
    
    kendall_control.add_y(profile.agreement_kendall())
    spearman_control.add_y(profile.agreement_spearman())
    tideman_control.add_y(Tideman(profile).agreement_tideman())
    
    for run in range(bootstrap_runs):
        p = profile.sample_bootstrap()
        kendall_control.add_sample(stage, p.agreement_kendall())
        spearman_control.add_sample(stage, p.agreement_spearman())
        tideman_control.add_sample(stage, Tideman(p).agreement_tideman())
    
    print("{}\t{}\t{}\t{}".format(
        stage,
        kendall_control.y()[stage],
        spearman_control.y()[stage],
        tideman_control.y()[stage]))

    
kendall_random = TimeSeriesResult()
spearman_random = TimeSeriesResult()
tideman_random = TimeSeriesResult()
print('\nRandom Pod')
print('stage\tkendall\tspearman\ttideman')
for stage, profile in enumerate(random_profiles):
    
    kendall_random.add_y(profile.agreement_kendall())
    spearman_random.add_y(profile.agreement_spearman())
    tideman_random.add_y(Tideman(profile).agreement_tideman())
    
    for run in range(bootstrap_runs):
        p = profile.sample_bootstrap()
        kendall_random.add_sample(stage, p.agreement_kendall())
        spearman_random.add_sample(stage, p.agreement_spearman())
        tideman_random.add_sample(stage, Tideman(p).agreement_tideman())
    
    print("{}\t{}\t{}\t{}".format(
        stage,
        kendall_random.y()[stage],
        spearman_random.y()[stage],
        tideman_random.y()[stage]))

In [ ]:
plt.errorbar(range(4), kendall_control.y(), yerr=kendall_control.yerr95(), label='Single Group', capsize=6)
plt.errorbar(range(4), kendall_random.y(), yerr=kendall_random.yerr95(), label='Random Pod', capsize=6)
plt.xticks(range(4))
plt.ylim(0,0.25)
plt.xlabel('Stage')
plt.ylabel('Kendall Correlation')
plt.grid()
plt.legend()
plt.title('Agreement (Kendall)')

In [ ]:
plt.errorbar(range(4), spearman_control.y(), yerr=spearman_random.yerr95(), label='Single Group', capsize=6)
plt.errorbar(range(4), spearman_random.y(), yerr=spearman_random.yerr95(), label='Random Pod', capsize=6)
plt.xticks(range(4))
plt.ylim(0,0.25)
plt.xlabel('Stage')
plt.ylabel('Spearman Correlation')
plt.grid()
plt.legend()
plt.title('Agreement (Spearman)')

In [ ]:
plt.plot(range(4), tideman_control.y(), label='Single Group')
plt.plot(range(4), tideman_random.y(),  label='Random Pod')
plt.xticks(range(4))
plt.xlabel('Stage')
plt.ylabel('Agreement (tideman)')
plt.grid()
plt.legend()
plt.title('Agreement (tideman)')

## Group-Level Results (Remove Attrition)

In [ ]:
df_score, proposal_map, proposal_rev_map = load_loomio_score('results/results_2_3.tsv')
df_score = remove_attrition(df_score)

treatments = {
    1: "Single Group",
    2: "Random Pod",
}

for treatment in [1, 2]:
    print("Treatment: {}".format(treatments[treatment]))
    stage_profiles = [
        Profile.from_score(df_score[(df_score.stage == stage) & (df_score.treatment == treatment)])
        for stage in sorted(set(df_score.stage))]

    Methods = [Majority, Borda, Tideman]
    print('stage\t' + '\t'.join([M.__name__ for M in Methods]))
    for stage, profile in enumerate(stage_profiles):
        winners = [repr(M(profile).social_choice()) for M in Methods]
        print(str(stage) + '\t' + '\t'.join(winners))
    print("")


### Group-Level Agreement (Remove Attrition)

In [ ]:
control_profiles = [
    Profile.from_score(df_score[(df_score.stage == stage) & (df_score.treatment == 1)])
    for stage in sorted(set(df_score.stage))]

random_profiles = [
    Profile.from_score(df_score[(df_score.stage == stage) & (df_score.treatment == 2)])
    for stage in sorted(set(df_score.stage))]

bootstrap_runs = 10

kendall_control = TimeSeriesResult()
spearman_control = TimeSeriesResult()
tideman_control = TimeSeriesResult()
print('Single Group')
print('stage\tkendall\tspearman\ttideman')
for stage, profile in enumerate(control_profiles):
    
    kendall_control.add_y(profile.agreement_kendall())
    spearman_control.add_y(profile.agreement_spearman())
    tideman_control.add_y(Tideman(profile).agreement_tideman())
    
    for run in range(bootstrap_runs):
        p = profile.sample_bootstrap()
        kendall_control.add_sample(stage, p.agreement_kendall())
        spearman_control.add_sample(stage, p.agreement_spearman())
        tideman_control.add_sample(stage, Tideman(p).agreement_tideman())
    
    print("{}\t{}\t{}\t{}".format(
        stage,
        kendall_control.y()[stage],
        spearman_control.y()[stage],
        tideman_control.y()[stage]))

    
kendall_random = TimeSeriesResult()
spearman_random = TimeSeriesResult()
tideman_random = TimeSeriesResult()
print('\nRandom Pod')
print('stage\tkendall\tspearman\ttideman')
for stage, profile in enumerate(random_profiles):
    
    kendall_random.add_y(profile.agreement_kendall())
    spearman_random.add_y(profile.agreement_spearman())
    tideman_random.add_y(Tideman(profile).agreement_tideman())
    
    for run in range(bootstrap_runs):
        p = profile.sample_bootstrap()
        kendall_random.add_sample(stage, p.agreement_kendall())
        spearman_random.add_sample(stage, p.agreement_spearman())
        tideman_random.add_sample(stage, Tideman(p).agreement_tideman())
    
    print("{}\t{}\t{}\t{}".format(
        stage,
        kendall_random.y()[stage],
        spearman_random.y()[stage],
        tideman_random.y()[stage]))

In [ ]:
plt.errorbar(range(4), kendall_control.y(), yerr=kendall_control.yerr95(), label='Single Group', capsize=6)
plt.errorbar(range(4), kendall_random.y(), yerr=kendall_random.yerr95(), label='Random Pod', capsize=6)
plt.xticks(range(4))
plt.ylim(0,0.25)
plt.xlabel('Stage')
plt.ylabel('Kendall Correlation')
plt.grid()
plt.legend()
plt.title('Agreement (Kendall)')

In [ ]:
plt.errorbar(range(4), spearman_control.y(), yerr=spearman_random.yerr95(), label='Single Group', capsize=6)
plt.errorbar(range(4), spearman_random.y(), yerr=spearman_random.yerr95(), label='Random Pod', capsize=6)
plt.xticks(range(4))
plt.ylim(0,0.25)
plt.xlabel('Stage')
plt.ylabel('Spearman Correlation')
plt.grid()
plt.legend()
plt.title('Agreement (Spearman)')

In [ ]:
plt.plot(range(4), tideman_control.y(), label='Single Group')
plt.plot(range(4), tideman_random.y(),  label='Random Pod')
plt.xticks(range(4))
plt.xlabel('Stage')
plt.ylabel('Agreement (tideman)')
plt.grid()
plt.legend()
plt.title('Agreement (tideman)')

## Group-Level Results (Forward-Fill Attrition)

In [ ]:
df_score, proposal_map, proposal_rev_map = load_loomio_score('results/results_2_3.tsv')
df_score = fill_attrition(df_score)

treatments = {
    1: "Single Group",
    2: "Random Pod",
}

for treatment in [1, 2]:
    print("Treatment: {}".format(treatments[treatment]))
    stage_profiles = [
        Profile.from_score(df_score[(df_score.stage == stage) & (df_score.treatment == treatment)])
        for stage in sorted(set(df_score.stage))]

    Methods = [Majority, Borda, Tideman]
    print('stage\t' + '\t'.join([M.__name__ for M in Methods]))
    for stage, profile in enumerate(stage_profiles):
        winners = [repr(M(profile).social_choice()) for M in Methods]
        print(str(stage) + '\t' + '\t'.join(winners))
    print("")


### Group-Level Agreement (Forward-Fill Attrition)

In [ ]:
control_profiles = [
    Profile.from_score(df_score[(df_score.stage == stage) & (df_score.treatment == 1)])
    for stage in sorted(set(df_score.stage))]

random_profiles = [
    Profile.from_score(df_score[(df_score.stage == stage) & (df_score.treatment == 2)])
    for stage in sorted(set(df_score.stage))]

bootstrap_runs = 10

kendall_control = TimeSeriesResult()
spearman_control = TimeSeriesResult()
tideman_control = TimeSeriesResult()
print('Single Group')
print('stage\tkendall\tspearman\ttideman')
for stage, profile in enumerate(control_profiles):
    
    kendall_control.add_y(profile.agreement_kendall())
    spearman_control.add_y(profile.agreement_spearman())
    tideman_control.add_y(Tideman(profile).agreement_tideman())
    
    for run in range(bootstrap_runs):
        p = profile.sample_bootstrap()
        kendall_control.add_sample(stage, p.agreement_kendall())
        spearman_control.add_sample(stage, p.agreement_spearman())
        tideman_control.add_sample(stage, Tideman(p).agreement_tideman())
    
    print("{}\t{}\t{}\t{}".format(
        stage,
        kendall_control.y()[stage],
        spearman_control.y()[stage],
        tideman_control.y()[stage]))

    
kendall_random = TimeSeriesResult()
spearman_random = TimeSeriesResult()
tideman_random = TimeSeriesResult()
print('\nRandom Pod')
print('stage\tkendall\tspearman\ttideman')
for stage, profile in enumerate(random_profiles):
    
    kendall_random.add_y(profile.agreement_kendall())
    spearman_random.add_y(profile.agreement_spearman())
    tideman_random.add_y(Tideman(profile).agreement_tideman())
    
    for run in range(bootstrap_runs):
        p = profile.sample_bootstrap()
        kendall_random.add_sample(stage, p.agreement_kendall())
        spearman_random.add_sample(stage, p.agreement_spearman())
        tideman_random.add_sample(stage, Tideman(p).agreement_tideman())
    
    print("{}\t{}\t{}\t{}".format(
        stage,
        kendall_random.y()[stage],
        spearman_random.y()[stage],
        tideman_random.y()[stage]))

In [ ]:
plt.errorbar(range(4), kendall_control.y(), yerr=kendall_control.yerr95(), label='Single Group', capsize=6)
plt.errorbar(range(4), kendall_random.y(), yerr=kendall_random.yerr95(), label='Random Pod', capsize=6)
plt.xticks(range(4))
plt.ylim(0,0.25)
plt.xlabel('Stage')
plt.ylabel('Kendall Correlation')
plt.grid()
plt.legend()
plt.title('Agreement (Kendall)')

In [ ]:
plt.errorbar(range(4), spearman_control.y(), yerr=spearman_random.yerr95(), label='Single Group', capsize=6)
plt.errorbar(range(4), spearman_random.y(), yerr=spearman_random.yerr95(), label='Random Pod', capsize=6)
plt.xticks(range(4))
plt.ylim(0,0.25)
plt.xlabel('Stage')
plt.ylabel('Spearman Correlation')
plt.grid()
plt.legend()
plt.title('Agreement (Spearman)')

In [ ]:
plt.plot(range(4), tideman_control.y(), label='Single Group')
plt.plot(range(4), tideman_random.y(),  label='Random Pod')
plt.xticks(range(4))
plt.xlabel('Stage')
plt.ylabel('Agreement (tideman)')
plt.grid()
plt.legend()
plt.title('Agreement (tideman)')